In [ ]:
import os
import shutil
import random
from PIL import Image

ModuleNotFoundError: No module named 'PIL'

In [ ]:


def create_split(image_folder, train_image_folder, val_image_folder):
    # Prüfen, ob die Zielordner existieren oder leer sind
    if not os.path.exists(train_image_folder) or not os.listdir(train_image_folder):
        os.makedirs(train_image_folder, exist_ok=True)
    else:
        print(f"Trainingsordner '{train_image_folder}' existiert bereits und ist nicht leer. Das Skript wird abgebrochen.")
        return

    if not os.path.exists(val_image_folder) or not os.listdir(val_image_folder):
        os.makedirs(val_image_folder, exist_ok=True)
    else:
        print(f"Validierungsordner '{val_image_folder}' existiert bereits und ist nicht leer. Das Skript wird abgebrochen.")
        return

    # Hole die Liste der Bilddateien im Quellordner
    image_files = [f for f in os.listdir(image_folder) if f.endswith(('.jpg', '.png'))]

    # Vergewissere dich, dass es tatsächlich Bilder gibt
    if len(image_files) == 0:
        print("Es gibt keine Bilddateien im Quellordner. Das Skript wird abgebrochen.")
        return

    # Zufällige Auswahl von 70% für Training und 30% für Validierung
    random.shuffle(image_files)

    split_index = int(0.7 * len(image_files))
    train_images = image_files[:split_index]
    val_images = image_files[split_index:]

    # Kopiere die Bilder in die entsprechenden Ordner
    for img in train_images:
        shutil.copy(os.path.join(image_folder, img), os.path.join(train_image_folder, img))

    for img in val_images:
        shutil.copy(os.path.join(image_folder, img), os.path.join(val_image_folder, img))

    print(f"Datensatz erfolgreich aufgeteilt: {len(train_images)} Bilder für Training, {len(val_images)} Bilder für Validierung.")

# Beispiel: Definiere deine Pfade
image_folder = "..\\Riseholme-2021\\Data\\Normal\\Ripe"
train_image_folder = "..\\Erdbeeren\\train_images"
val_image_folder = "..\\Erdbeeren\\val_images"

# Führe die Funktion aus
create_split(image_folder, train_image_folder, val_image_folder)


Datensatz erfolgreich aufgeteilt: 323 Bilder für Training, 139 Bilder für Validierung.


In [ ]:
def create_yolo_labels(image_folder, label_folder):
    """
    Erstellt YOLO-kompatible Label-Dateien (TXT) für jedes Bild im angegebenen Ordner.
    Angenommen wird, dass sich jeweils eine Erdbeere auf dem Bild befindet,
    und die Bounding Box das komplette Bild abdeckt oder manuell festgelegt wird.

    Parameters:
    - image_folder (str): Pfad zu den Bildern
    - label_folder (str): Pfad zum Speicherort der Label-Dateien
    """

    if not os.path.exists(label_folder):
        os.makedirs(label_folder)

    # Alle Dateien im Bildordner durchlaufen
    for image_name in os.listdir(image_folder):
        if image_name.lower().endswith(('.jpg', '.jpeg', '.png')):
            image_path = os.path.join(image_folder, image_name)
            image = Image.open(image_path)
            width, height = image.size

            # Wir nehmen an: eine Erdbeere = ein Objekt, ganze Bildgröße = Bounding Box
            # (alternativ könntest du manuell oder mit einem Tool genaue Boxen setzen)

            # YOLO erwartet: class x_center y_center width height (alles relativ)
            x_center = 0.5
            y_center = 0.5
            rel_width = 1.0
            rel_height = 1.0

            label_filename = os.path.splitext(image_name)[0] + '.txt'
            label_path = os.path.join(label_folder, label_filename)

            with open(label_path, 'w') as f:
                f.write(f"0 {x_center} {y_center} {rel_width} {rel_height}\n")

            print(f"[✓] Label geschrieben: {label_path}")


In [ ]:
train_image_folder = "..\\Erdbeeren\\train_images"
train_label_folder = "..\\Erdbeeren\\train_labels"

val_image_folder = "..\\Erdbeeren\\val_images"
val_label_folder = "..\\Erdbeeren\\val_labels"

# YOLO-Labeldateien erzeugen
create_yolo_labels(train_image_folder, train_label_folder)
create_yolo_labels(val_image_folder, val_label_folder)
